# Import

In [376]:
import numpy as np
from random import choice, choices, randint,random, uniform
import copy
import math
from dataclasses import dataclass

# Operations
Operations divided based on the arity in order to make more consistent program generation during initialization, avoiding excessive nesting of trigonometric functions.

Also the max depth of the tree is set in order to avoid bloating, a problem that mainly exists because of the crossover and causes the formula to grow a lot without real changes into fitness

In [377]:
#The depth of a tree cannot be greater than 10 OVERALL
MAX_TREE_DEPTH = 10

"""Field of probabilities not used anymore, in order not to change implementation, we left it in a way that they are all equal to 0.1"""
OPERATIONS_BINARY = [ 
    (np.add, 2, "({} + {})",0.1),
    (np.subtract, 2, "({} - {})",0.1),
    (np.divide, 2, "({} / {})",0.1),
    (np.multiply, 2, "({} * {})",0.1),   
    (np.power, 2, "({} ^ {})",0.1),
]

BINARY_WEIGHTS = [op[3] for op in OPERATIONS_BINARY]

OPERATIONS_UNARY = [ 
    (np.sin, 1, "sin({})",0.1),
    (np.cos, 1, "cos({})",0.1),
    (np.exp, 1, "exp({})",0.1),
    (np.log, 1, "log({})",0.1),
    (np.sqrt, 1, "sqrt({})",0.1),
]
UNARY_WEIGHTS = [op[3] for op in OPERATIONS_UNARY]
OPERATIONS = OPERATIONS_BINARY + OPERATIONS_UNARY
WEIGHTS = BINARY_WEIGHTS + UNARY_WEIGHTS    


# Program evaluation
We need a function that, given the genotype, provides us with the numerical output given by the input applied to the function. Thus, it has to receive as arguments the program (that represents the function) and the input vector with all the dimensions. Then, it simply recursively travels the tree and compute all the operations.

In [378]:
def evaluate_program(program, x):
    if isinstance(program, str):  # Leaf node
        #If it's a leaf, it could be a costant or a variable
        if program[0] == 'x':
            return x[int(program[2:-1])]
        else:
            return float(program)
    elif isinstance(program, list): 
        op = next(op for op, _, symbol,p in OPERATIONS if symbol == program[0])
        args = [evaluate_program(child, x) for child in program[1:]]
        try:
            return op(*args)
        except ZeroDivisionError:
            return np.inf

# Mutation utility functions

Set of utility functions used in the mutation and crossover functions. 

In [379]:

def get_subtree_points_recursive(prog, path='', index=0, result=None):
    """ Get the points of the subtrees in the program. """
    if result is None:
        result = []
    
    if isinstance(prog, list) and len(prog) > 1:  
        if path:
            result.append(path)
        for i, node in enumerate(prog):
            if isinstance(node, list):  
                new_path = f"{path}[{i}]" if path else f"[{i}]"
                get_subtree_points_recursive(node, new_path, i, result)
    return result

def access_node_by_path(prog, path):
    """Access a node in the program by its path."""
    indices = [int(p.strip('][')) for p in path.split('][') if p]
    current = prog
    for index in indices:
        current = current[index]
    return current


def find_variable_indices(node, result=None):
    """Find the indices of the variables in the program."""
    if result is None:
        result = set()

    if isinstance(node, list):
        for child in node:
            find_variable_indices(child, result)
    
    elif isinstance(node, str) and node.startswith('x['):
        result.add(int(node[2:-1]))

    return result

def set_subtree_at_path(program, path, new_subtree):
    """Set the new subtree at the specified path."""
    if path == '':
        return new_subtree
    current = program
    indices = [int(x) for x in path.strip('][').split('][')]
    for i in indices[:-1]:  
        current = current[i]
    current[indices[-1]] = new_subtree  
    return program


def swap_operation_at_path(program, path, new_op):
    """Swap the operation at the specified path."""
    current = program
    indices = [int(x) for x in path.strip('][').split('][')]
    for i in indices[:-1]: 
        current = current[i]
    if isinstance(current[indices[-1]], list):  
        current[indices[-1]][0] = new_op 
    return program

def cleaned_program(program):
    """Given a program of only costants, substitute it with a single value given by the computation of costants"""
    value = evaluate_program(program, [0])
    return str(value)

def safe_copy(obj):
    """Copy an object safely."""
    if isinstance(obj, list):
        return copy.deepcopy(obj)  
    elif isinstance(obj, str):
        return obj 
    else:
        raise TypeError("Unsupported type, only strings.")
    
def depth(program):
    """Compute the depth of the program (represented by a tree)"""
    if isinstance(program, str):
        return 1
    elif isinstance(program, list):
        return 1 + max(depth(child) for child in program[1:])
    
def get_subtree(program):
    """Get a random subtree from the program."""
    if isinstance(program, (str, int, float)):
        return program, None
    # Pick a random node
    scelta = randint(0, len(program) - 1)
    if scelta == 0:
        # Return the entire program
        return program, None
    else:
        # Recursion on one of the children
        subtree, parent = get_subtree(program[scelta])
        return subtree, (program, scelta)

# Genotype definition

The recursive function we've developed is designed to create complex mathematical formulas in a structured format, where each formula is represented as a list: `[operator, [first_operand], [second_operand]]` or `[operator, [operand]]` if it only takes one argument. This format allows the recursive and hierarchical organization of operations, facilitating the computational evaluation and manipulation of the formula.

Key features of this function include:

1. **Use of Input Dimensions:** The function uses a list, `used_indices`, to track which input dimensions (e.g., `0,1`, etc.) have been utilized in the formula. This ensures comprehensive coverage of all available input variables (if the depth makes it possible), making the formula relevant to all dimensions of the input data.  

2. **Restrictions on Trigonometric Functions:** To enhance the mathematical sensibility of the formulas generated, the function includes a specific constraint regarding the nesting of trigonometric functions. Once a trigonometric function (such as `sin`, `cos`, etc.) is used, the function prohibits the inclusion of another trigonometric function within it. This constraint helps in preventing mathematically nonsensical expressions like `sin(cos(tan(x)))`, which, while computationally valid, may not be practically meaningful or may complicate the interpretation and analysis of the formula.
3. **Restrictions on operations with constants:** It is not allowed, in the current implementation, to add binary operations between constants. This is another kind of operation that makes the depth of the tree increase without the generation of a real meaningful formula.

This approach not only ensures that each formula is robust and contextually appropriate but also maintains clarity and reduces the computational redundancy that might arise from nested trigonometric operations. Such constraints are particularly important in scientific computing and simulations where the accuracy and interpretability of mathematical expressions are critical.



In [380]:
def random_program(depth,input_dim,unary=False, used_indices=None):
    if used_indices is None:
        used_indices = set()

    # Base case: generate a leaf node
    #If you already used all the variables, you can only use constants and return: don't place an operation between costants
    if depth == 0 or (random()<0.3) or len(used_indices) == input_dim:
        if len(used_indices) < input_dim and random()<0.7:
            available_indices = list(set(range(input_dim)) - used_indices)
            index = choice(available_indices)
            used_indices.add(index)
            return f"x[{index}]", used_indices
        else: 
            #Only positive float between 0 and 10, no need to include negative ones, we have the subtraction operation
            return str(uniform(2, 10)), used_indices
     #Condition to check that we do not nest unary operations     
    if(not unary):
        operations = OPERATIONS
        weights = WEIGHTS
    else:
        operations = OPERATIONS_BINARY
        weights = BINARY_WEIGHTS
    op, arity, symbol,p = choices(operations,weights=weights, k=1)[0]
    if(unary != True):
        unary = arity==1
    children = []
    for _ in range(arity):
        child, used_indices = random_program(depth - 1, input_dim, unary, used_indices)
        children.append(child)
    return [symbol] + children,used_indices

# Program generation for the initial population
This function, given the input dimension (the number of variables), generates a program that contains all of them once by generating one subtree for each dimension and combining all the subtrees together. This allows an initial population where there is a lot of genetic material related to all the dimensions of the problem and we can create more different combinations. 

In [381]:
def generate_program(input_dim):
    """Generate a random program given the input dimension. It ensures all dimensions are used once in the program."""
    programs = []
    for i in range(input_dim):
        flag = False #Evaluate that it does not only contain costants
        used_indices_local = set()
        used_indices_local = set(range(input_dim)) - {i}
        while(not flag):
            program, _ = random_program(2, input_dim,used_indices=used_indices_local)
            flag = len(find_variable_indices(program)) == 1
            if not flag:
                program = cleaned_program(program)
        programs.append(program)
    #Combine programs together through a binary operation
    program = programs[0]
    for i in range(1,len(programs)):
        op, arity, symbol,p = choices(OPERATIONS_BINARY,weights=BINARY_WEIGHTS, k=1)[0]
        program = [symbol] + [program, programs[i]]
    
    return program

## Individual

We used dataclass to store the fitness and the genome of each individual. This will allow us to avoid to recompute the fitness function for the same individual more than once.

In [382]:
@dataclass
class Individual:
    genome: list
    fitness : float = None

## Transform the program into a human readable function
Given the list representation of programs that we have introduced before, this function allows to rewrite it in a human readable way by simply recursively evaluating all nodes.

In [383]:
def program_to_string(program):
    if isinstance(program, str):  # leaf
        return program  
    elif isinstance(program, list):  
        try:
            _, _, symbol = next((op, arity, s) for op, arity, s,p in OPERATIONS if s == program[0])
        except StopIteration:
            raise ValueError(f"Not known operation: {program[0]}")
        
        children = [program_to_string(child) for child in program[1:]]
        
        return symbol.format(*children)

## Fitness function

# MAPE
The Mean Absolute Percentage Error (MAPE) is a statistical measure used to evaluate the accuracy of predictions in regression analysis. It quantifies the average magnitude of errors in a set of predictions, expressed as a percentage of the actual values. The formula for MAPE is the average of the absolute differences between the predicted and actual values, divided by the actual values, multiplied by 100 to convert it into a percentage. This metric is particularly useful because it provides a clear, interpretable measure of predictive error relative to the size of the numbers being predicted, making it ideal for comparing the accuracy of prediction models across different data scales

In [384]:
def mape(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return  float(np.mean(np.abs((y - predictions) / y)) * 100)

# MSE 
We used MAPE only for printing it and having an idea of how the algorithm is evolving the formula, the fitness function associated to the individual is the mse. Indeed, mse and mape do not vary in a consistent way. There are functions that decreases the mse while increasing the mape, and this can be dued in particular to the presence of 'outliers', which are values that reports a huge different with respect to the real value and count more in the percentage with respect to the mse.

Since the final evaluation on the quality of the work is done based on the mse, we decided to keep it as fitness. 

However, since the problems are on a different scale, the mse may be misleading: it is higher in some problems because the y reports high values so, even if the difference between the prediction and the real value is not so high in percentage, the mse is high. That's why we also printed mape to have an idea regarding the quality of the algorithm when doing experimentations.

In [385]:
def mse(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return float(np.mean((y - predictions) ** 2))

In [386]:
def fitness_function(program, x, y):
    """Fitness evaluation for a program."""
    try:
        # Evaluation of the program
        predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
        if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
            return np.inf  # Penalize invalid programs 

        #Compute mse
        error = mse(program, x, y)
       
        if not math.isfinite(error):
            return np.inf
        

    except Exception as e:
        # Penalize programs with errors 
        print(f"Error in program evaluation: {e}")
        return np.inf

    return error

# Avoiding bloating
In order to avoid bloating, which means that programs keep growing without a significant change in fitness, we also introduced a function to cut the tree when it reaches a depth that is greater than MAX_TREE_DEPTH. 

In [387]:
def cut_program(program, current_depth=0):
    """Cut the program in a way that the depth is at most MAX_TREE_DEPTH."""
    if isinstance(program, str):
        return program  # Leafs are mantained
    elif isinstance(program, list):
        operation = program[0]
        if current_depth > MAX_TREE_DEPTH:
            return str(uniform(2, 10))
        else:
            return [operation] + [cut_program(subprogram, current_depth + 1) for subprogram in program[1:]]


## Mutation
Simply mutates a subtree with another one randomly generated -> subtree mutation

In [388]:
def mutate(program, input_dim, max_depth=2):
    """Mutation of a program."""
    mutant = program #the deep copy is done when passing the program when the function is called
    #If the program has a depth lower equal than 2, increase it
    if(depth(program)<=4 and random()<0.6):
        #randomly select a binary operation
        _, _, symbol,_ = choices(OPERATIONS_BINARY,weights=BINARY_WEIGHTS, k=1)[0]
        #generate a program
        new_subtree = generate_program(input_dim)
        #Combine the two programs
        return [symbol] + [program, new_subtree]
    points = get_subtree_points_recursive(program)
    if not points:
        return generate_program(input_dim)
    
    point = choice(points)
    new_subtree = random_program(randint(1,max_depth), input_dim)[0]

    mutant = set_subtree_at_path(mutant, point, new_subtree)
    if(depth(mutant)>MAX_TREE_DEPTH):
        mutant = cut_program(mutant)
    return mutant

# Hoist mutation

1. **Select a Subtree**:
   - A random subtree of the program is selected using the helper function `get_subtree`.

2. **Check if the Subtree is a Leaf**:
   - If the selected subtree is a leaf (e.g., a variable or constant), the program is returned unchanged since hoisting cannot be applied to a single node.

3. **Select an Inner Subtree**:
   - A smaller subtree within the selected subtree is identified using `get_subtree`.

4. **Replace the Parent Node**:
   - If the selected subtree is the root of the program:
     - The entire program is replaced with the inner subtree.
   - Otherwise:
     - The parent of the selected subtree is updated to reference the inner subtree.

In [389]:
def hoist_mutation(program):
    """Mutation of a program through hoist"""
    # Obtein a random subtree
    subtree, parent_info = get_subtree(program)
    
    if not isinstance(subtree, list):
        # If the subtree is a leaf, return the program, hoist cannot be done
        return program

    # Get the inner subtree
    inner_subtree, _ = get_subtree(subtree)

    if parent_info is None:
        # Sobstitute the program with the inner subtree
        return inner_subtree
    else:
        # Sobstitute the parent with the inner subtree
        parent, idx = parent_info
        parent[idx] = inner_subtree
        return program

# Point mutation

The mutation involves randomly modifying a node in the program, which could be an operator, variable, or constant, while ensuring that the resulting program remains syntactically valid.

Key Features
1. Ensures structural integrity by considering the arity of operators during mutations.
2. Limits program depth to avoid overgrowth.
3. Maintains syntactic validity by mutating nodes in a context-aware manner.
4. Customizable mutation probability for operators (30% by default).

In [391]:
def point_mutation(program, max_tree_depth=MAX_TREE_DEPTH):
    """
    Perform a point mutation on the given program.

    Parameters:
    - program: list
        The symbolic program represented as a nested list (tree structure).
    - max_tree_depth: int
        Maximum depth allowed for the tree.

    Returns:
    - mutated_program: list
        A new program with a randomly mutated node.
    """

    #let's define a function that tell us the variables inside the program:
    def collect_variables(node, variables):
        """Collect all variable names in the program."""
        if isinstance(node, str) and node.startswith('x'):
            variables.add(node)
        elif isinstance(node, list):
            for child in node:
                collect_variables(child, variables)
        

    def mutate_leaf(node):
        """Mutate a leaf node (constant or variable)."""
        if isinstance(node, str):
            # Variable mutation: Switch to another variable or a random constant that is present in 
            #the current program:
            if node.startswith('x'):
                # Collect variables from the program.
                variables = set()
                collect_variables(program, variables)
                return choice(list(variables))
            else:
                #Replace with a new random constant.
                #for now i consider only between -10 and 10, but we can change it
                return str(round(uniform(-10, 10), 2))
        return node

    def mutate_operator(node):
        """Mutate an operator node."""
        arity = len(node) - 1
        #perations by matching arity
        valid_ops = [op for op in OPERATIONS if op[1]==arity]
    
        new_op = choice(valid_ops)
        node[0] = new_op[2]  # Replace operator symbol.

    def recursive_mutation(node, current_depth):
        """Recursively traverse the tree and apply mutation."""
        if current_depth >= max_tree_depth or not isinstance(node, list):
            # Mutate a leaf node if max depth reached or node is a leaf.
            return mutate_leaf(node)

        if isinstance(node, list):
            # Decide whether to mutate this operator or recurse further
            if random() < 0.3:  # 30% chance to mutate this operator but if yoi prefer we can change
                mutate_operator(node)
            else:
                # Recursively mutate a random child
                child_idx = randint(1, len(node) - 1)
                node[child_idx] = recursive_mutation(node[child_idx], current_depth + 1)

        return node

    mutated_program = copy.deepcopy(program)
    return recursive_mutation(mutated_program, current_depth=0)

## Crossover

We can use a croossover function that receives only 2 parents and, if one of them is a leaf program simply return casually one of the 2 programs (avoiding to perform the operation for programs with no childrens). Otherwise, select random indexes for both the parents and combine the first part of the tree with the second part of the tree of the 2 parents, returning a new individual.

In order to avoid bloating, the tree is cut if its depth is greater than MAX_TREE_DEPTH.

In [392]:
def crossover(parent1, parent2):
    """Crossover by subtree swapping."""
    #Obtain a casual subtree from parents
    subtree1, parent1_info = get_subtree(parent1)
    subtree2, _ = get_subtree(parent2)

    if parent1_info is None:
        # A subtree of parent2 is returned, no risk of having more than MAX_TREE_DEPTH
        return subtree2
    else:
        # Subtree swapping and check on the final depth
        parent, idx = parent1_info
        parent[idx] = subtree2
        if(depth(parent)>MAX_TREE_DEPTH): #Cut the program only if necessary
            return cut_program(parent) 
        return parent

## Genetic algorithm

### Data loading

In [393]:
#load the problem with problem_X, for X that goes from 0 to 8
problem = np.load('data/problem_7.npz')
x = problem['x']
y = problem['y']
input_dim = x.shape[0]

## Tournament selection for parents with fitness hole, tau=10
Tournament selection is used in order to select an individual over a population. 
This algorithm, given the population, selects 10 random individuals and performs a tournament among them. There is a probability of 90% that the fittest individual wins, while, the remaining 10% of times, the opposite is done. 
Implementing a fitness hole as described can actually be beneficial in overcoming the challenges posed by an adaptive change that requires multiple intermediate steps. By intentionally allowing less fit individuals a chance to win, fitness holes can help navigate the evolutionary pathway where direct progression is hindered by intermediate steps that reduce overall fitness. This approach ensures that even though the final adaptation is advantageous, the evolutionary path to achieve it can successfully bypass 'fitness holes' that would otherwise deselect the intermediates before the final adaptation is achieved.

In [394]:
def tournament_selection(population,tau=10):
    tau = min(tau, len(population)) #not needed in theory
    tournament_indices = np.random.choice(len(population), tau, replace=False)

    considered_individuals = []
    for index in tournament_indices:
        considered_individuals.append(population[index])
    considered_individuals.sort(key=lambda i: i.fitness)
    if random() < 0.9:
        winner = considered_individuals[0].genome
    else:
        #Select the worst individual
        winner = considered_individuals[-1].genome
    return winner


In [395]:
def initialize_and_evaluate():
    """Generate a random individual and evaluate its fitness."""
    individual = Individual(genome=generate_program(input_dim))
    individual.fitness = fitness_function(individual.genome, x, y)
    return individual

## Parameters

In [396]:
generations = 50
population_size = 1000
p_crossover = 0.7
p_subtree_mutation = 0.2
p_hoist = 0.05
offspring_size = population_size 

# Steady-state evolutionary algorithm

In [ ]:
# Avoid printing warnings
np.seterr(all='ignore')

# Initialize the population and sort it 
population = [initialize_and_evaluate() for _ in range(population_size)]
population.sort(key=lambda i: i.fitness)

def run_genetic_algorithm():
    global population

    for gen in range(generations):
        
        mape_val = mape(population[0].genome, x, y)
        mse_val = mse(population[0].genome, x, y)
        np.seterr(all='warn')
        print(f"Generation {gen + 1}, mape: {mape_val}, mse: {mse_val}, depth: {depth(population[0].genome)}")
        #population is already sorted, so:
        print(f"Best formula: {program_to_string(population[0].genome)}")
        np.seterr(all='ignore')
    
        next_population = []
        
        while len(next_population) < offspring_size:
            if random() < p_crossover:
                # Crossover
                parent1, parent2 = tournament_selection(population), tournament_selection(population)
                child1 = crossover(safe_copy(parent1), safe_copy(parent2))      
                next_population.append(Individual(genome=child1, fitness=fitness_function(child1, x, y)))
            else:
                parent = tournament_selection(population)
                if(random()< p_subtree_mutation):
                    mutant = mutate(safe_copy(parent),input_dim)
                else:
                    mutant = point_mutation(safe_copy(parent)) 
                if(random()<p_hoist):
                    mutant = hoist_mutation(safe_copy(mutant))
                next_population.append(Individual(genome=mutant, fitness=fitness_function(mutant, x, y)))
                    
        # the new population is the one generated in the offspring
        population.extend(next_population)

        # Remove duplicates
        unique_population = {}
        for prog in population:
            serialized = str(prog)
            if serialized not in unique_population:
                unique_population[serialized] = prog
        
        # update fitness of the new population
        population = list(unique_population.values())
        
        population.sort(key=lambda i: i.fitness)
        population = population[:population_size]
        
    best_program = population[0]
    return best_program

best_program = run_genetic_algorithm()
print("Best program:", best_program.genome, "; Fitness:", best_program.fitness)

Generation 1, mape: 272.7893382107887, mse: 687.6146500210604, depth: 4
Best formula: ((x[0] * 3.6696762445215283) * (sin(x[1]) * 2.311184441053867))
Generation 2, mape: 221.20343556185662, mse: 658.6101210431474, depth: 5
Best formula: (((x[0] * 3.6696762445215283) * (sin(x[1]) * 2.311184441053867)) + log((x[1] + 4.825498965674644)))
Generation 3, mape: 79.3190392851772, mse: 623.4783945166375, depth: 4
Best formula: (((x[0] + 5.603241806391207) - 4.224452726715239) * ((3.3047325342718423 + x[0]) ^ x[1]))
Generation 4, mape: 66.69004464585285, mse: 364.37257579584565, depth: 3
Best formula: (4.16209945259306 ^ (x[0] * x[1]))
Generation 5, mape: 66.69004464585285, mse: 364.37257579584565, depth: 3
Best formula: (4.16209945259306 ^ (x[0] * x[1]))
Generation 6, mape: 182.18628216674784, mse: 346.8243515784463, depth: 7
Best formula: (((4.16209945259306 ^ (x[0] * x[1])) - ((log(6.5538608664522) * x[0]) + x[1])) + (6.04 + (((x[0] * 3.6696762445215283) * (sin(x[1]) * 2.311184441053867)) + e

In [398]:
program_to_string(best_program)

'((((((((4.16209945259306 ^ (x[0] * x[1])) + ((8.050256961346829 * 9.495403289272176) / 6.8136662010600375)) + 6.122674953917595) ^ cos((x[1] - x[0]))) * 1.73) ^ cos((x[1] - x[0]))) ^ cos((x[1] - x[0]))) ^ cos((x[1] - x[0])))'